# Data Analysis of the Rising Cost of Rent in American College Towns

## _Data Wrangling Part 1_

## About the data
This notebook utilizes Fair Market Rent Data from the U.S. Department of Housing and Urban Development (HUD)'s Office of Office of Policy Development and Research (https://www.huduser.gov/portal/datasets/fmr.html#history).

## Background on the data
Here are the meanings of some of the columns present in the data:
- `areaname`: geographic area name
- `cntyname`: county name
- `pmsaname`: primary metropolitan statistical area name
- `fmrxx_y`: xx represents year; y represents number of bedrooms ranging from 0(efficiency) to 4 bedrooms
- `fmr_area`: shows the fmr percentile measured
- `pop2017`: estimated population 2017
- `pop2000`: population from 2000 census

### First, we import data and start to clean

In [1]:
import numpy as np
import pandas as pd

In [2]:
rent_prices = pd.read_csv('rent_prices.csv')
rent_prices.head(3)

,fips2010,fips2000,areaname22,name,msa22,fmr22_0,fmr22_1,fmr22_2,fmr22_3,fmr22_4,...,pop2010,fmr_area,census_region,pmsaname,cntyname,pop2017,pop2000,id_agis3,id_agis2,id
0,100199999.0,100199999.0,"Montgomery, AL MSA",Autauga County,METRO33860M33860,643.0,764.0,914.0,1156.0,1494.0,...,54571.0,5240.0,3.0,"Montgomery, AL MSA",Autauga County,55035.0,43671.0,MSA5240,MSA5240,100000001.0
1,100399999.0,100399999.0,"Daphne-Fairhope-Foley, AL MSA",Baldwin County,METRO19300M19300,772.0,777.0,1017.0,1348.0,1715.0,...,182265.0,5160.0,3.0,"Mobile, AL MSA",Baldwin County,203360.0,140415.0,MSA5160,MSA5160,100000003.0
2,100599999.0,100599999.0,"Barbour County, AL",Barbour County,NCNTY01005N01005,532.0,536.0,705.0,871.0,980.0,...,27457.0,10000005.0,3.0,"Barbour County, AL",Barbour County,26200.0,29038.0,CNTY01005,CNTY01005,100000005.0


### Delete unneeded rows

We only need the district information and the historical rent prices. The rent prices columns all start with fmr. We need to see all the columns that don't start with fmr so that we can know the columns that will be dropped.

In [3]:
to_be_dropped = [columns for columns in rent_prices.columns if not (columns.startswith('fmr'))]
print (to_be_dropped)

['fips2010', 'fips2000', 'areaname22', 'name', 'msa22', 'msa21', 'msa20', 'msa19', 'msa18', 'msa17', 'msa16', 'msa15', 'msa14', 'msa13', 'msa12', 'msa11', 'msa10', 'msa09', 'msa08', 'msa07', 'msa06', 'msa05', 'msa04', 'msa03', 'msa02', 'msa01', 'msa00', 'msa99', 'msa98', 'msa97', 'msa96', 'msa95', 'msa94', 'msa93', 'msa92', 'msa91', 'msa90', 'msa89', 'msa88', 'msa87', 'msa86', 'msa85', 'msa83', 'cbsasub', 'areaname', 'state', 'cousub', 'msa', 'county', 'pop2010', 'census_region', 'pmsaname', 'cntyname', 'pop2017', 'pop2000', 'id_agis3', 'id_agis2', 'id']


Let's take a peep at the columns that we want to drop

In [4]:
rent_prices[['areaname','state','cousub','county','census_region',
             'pmsaname','cntyname','areaname22','name','cbsasub']].head()

,areaname,state,cousub,county,census_region,pmsaname,cntyname,areaname22,name,cbsasub
0,"Montgomery, AL MSA",1.0,99999.0,1.0,3.0,"Montgomery, AL MSA",Autauga County,"Montgomery, AL MSA",Autauga County,METRO33860M33860
1,"Daphne-Fairhope-Foley, AL MSA",1.0,99999.0,3.0,3.0,"Mobile, AL MSA",Baldwin County,"Daphne-Fairhope-Foley, AL MSA",Baldwin County,METRO19300M19300
2,"Barbour County, AL",1.0,99999.0,5.0,3.0,"Barbour County, AL",Barbour County,"Barbour County, AL",Barbour County,NCNTY01005N01005
3,"Birmingham-Hoover, AL HUD Metro FMR Area",1.0,99999.0,7.0,3.0,"Bibb County, AL",Bibb County,"Birmingham-Hoover, AL HUD Metro FMR Area",Bibb County,METRO13820M13820
4,"Birmingham-Hoover, AL HUD Metro FMR Area",1.0,99999.0,9.0,3.0,"Birmingham, AL MSA",Blount County,"Birmingham-Hoover, AL HUD Metro FMR Area",Blount County,METRO13820M13820


We want to keep name, cntynme and areaname columns because they are all different identifiers for a region. This will come in handy when we compare with other datasets.

In [5]:
rent_prices.drop(columns = [col for col in rent_prices.columns if (col.startswith(('fips','msa','id','pop')))],inplace=True)
rent_prices.drop(columns = ['state','cousub','county','census_region','areaname22','cbsasub','name'],inplace=True)

The fmr columns that don't have _ represent fmr percentile and are not needed. The fmr_area column is an efficiency measurement that is not needed. We shall try to delete them before proceeding.

In [6]:
rent_prices.drop(columns = [col for col in rent_prices.columns if col.startswith('fmr') if len(col)==5], inplace=True)
rent_prices.drop(columns = 'fmr_area', inplace=True)
rent_prices.head()

,fmr22_0,fmr22_1,fmr22_2,fmr22_3,fmr22_4,fmr21_0,fmr21_1,fmr21_2,fmr21_3,fmr21_4,...,fmr85_3,fmr85_4,fmr83_0,fmr83_1,fmr83_2,fmr83_3,fmr83_4,areaname,pmsaname,cntyname
0,643.0,764.0,914.0,1156.0,1494.0,640.0,766.0,908.0,1148.0,1520.0,...,344.0,382.0,186.0,227.0,269.0,332.0,370.0,"Montgomery, AL MSA","Montgomery, AL MSA",Autauga County
1,772.0,777.0,1017.0,1348.0,1715.0,718.0,723.0,922.0,1249.0,1584.0,...,393.0,439.0,217.0,257.0,309.0,380.0,425.0,"Daphne-Fairhope-Foley, AL MSA","Mobile, AL MSA",Baldwin County
2,532.0,536.0,705.0,871.0,980.0,488.0,492.0,648.0,806.0,907.0,...,387.0,426.0,212.0,257.0,300.0,374.0,413.0,"Barbour County, AL","Barbour County, AL",Barbour County
3,765.0,820.0,943.0,1220.0,1316.0,817.0,871.0,1002.0,1303.0,1409.0,...,400.0,447.0,218.0,265.0,312.0,387.0,433.0,"Birmingham-Hoover, AL HUD Metro FMR Area","Bibb County, AL",Bibb County
4,765.0,820.0,943.0,1220.0,1316.0,817.0,871.0,1002.0,1303.0,1409.0,...,417.0,462.0,229.0,280.0,327.0,404.0,448.0,"Birmingham-Hoover, AL HUD Metro FMR Area","Birmingham, AL MSA",Blount County


### Change column names

We should change the fmr columns into dates to better visualize the data

In [7]:
old_col = rent_prices.columns
new_col = ['20'+i[-4:] for i in [col for col in rent_prices.columns if col.startswith('fmr2')]]\
         +['19'+i[-4:] for i in [col for col in rent_prices.columns if col.startswith('fmr9')]]\
         +['19'+i[-4:] for i in [col for col in rent_prices.columns if col.startswith('fmr8')]]\
         +['20'+i[-4:] for i in [col for col in rent_prices.columns if col.startswith('fmr1')]]\
         +['20'+i[-4:] for i in [col for col in rent_prices.columns if col.startswith('fmr0')]]

rent_prices.rename(columns = dict(zip(old_col,new_col)),inplace=True)

In [8]:
rent_prices.sort_index(axis=1,ascending=False,inplace=True)
rent_prices.head(1)

,pmsaname,cntyname,areaname,2022_4,2022_3,2022_2,2022_1,2022_0,2021_4,2021_3,...,1985_4,1985_3,1985_2,1985_1,1985_0,1983_4,1983_3,1983_2,1983_1,1983_0
0,"Montgomery, AL MSA",Autauga County,"Montgomery, AL MSA",1494.0,1156.0,914.0,764.0,643.0,1520.0,1148.0,...,1020.0,776.0,583.0,517.0,440.0,881.0,731.0,537.0,454.0,425.0


### Identify null values

We need to check the data for null values and see if the summary statistics make sense

In [9]:
rent_prices.describe(include='all')

,pmsaname,cntyname,areaname,2022_4,2022_3,2022_2,2022_1,2022_0,2021_4,2021_3,...,1985_4,1985_3,1985_2,1985_1,1985_0,1983_4,1983_3,1983_2,1983_1,1983_0
count,4757,4757,4766,4765.000000,4765.000000,4765.000000,4765.000000,4767.000000,4766.000000,4766.000000,...,4738.000000,4738.000000,4738.000000,4740.000000,4738.000000,4736.000000,4736.000000,4736.000000,4736.000000,4738.000000
unique,2674,1959,2598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,"Boston, MA--NH PMSA",Washington County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,129,104,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,1481.819098,1288.199161,991.092760,793.927177,724.395930,1436.278011,1251.494964,...,887.020473,784.441537,608.495990,502.509156,446.455255,885.768370,767.869299,591.293708,473.361909,397.414268
std,NaN,NaN,NaN,503.423202,433.910600,348.472099,287.402187,256.837765,514.150816,447.021624,...,295.683966,250.893247,204.548023,171.364501,154.539266,349.519727,293.129406,228.428755,183.796258,157.456783
min,NaN,NaN,NaN,574.000000,549.000000,422.000000,370.000000,356.000000,571.000000,524.000000,...,408.000000,396.000000,307.000000,272.000000,215.000000,431.000000,390.000000,309.000000,263.000000,213.000000
25%,NaN,NaN,NaN,1122.000000,1000.000000,757.000000,600.000000,555.000000,1085.000000,972.000000,...,681.250000,605.000000,467.000000,385.000000,342.000000,630.000000,556.000000,436.000000,346.000000,296.000000
50%,NaN,NaN,NaN,1313.000000,1130.000000,867.000000,699.000000,639.000000,1260.000000,1084.000000,...,802.000000,707.000000,537.000000,446.500000,407.000000,778.000000,675.000000,531.000000,415.000000,353.000000
75%,NaN,NaN,NaN,1729.000000,1472.000000,1122.000000,884.000000,811.500000,1648.000000,1404.000000,...,1035.000000,897.000000,682.000000,558.000000,499.000000,1037.750000,892.000000,678.000000,539.000000,439.000000


In [10]:
rent_prices.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 198 columns):
 #    Column    Non-Null Count  Dtype  
---   ------    --------------  -----  
 0    pmsaname  4757 non-null   object 
 1    cntyname  4757 non-null   object 
 2    areaname  4766 non-null   object 
 3    2022_4    4765 non-null   float64
 4    2022_3    4765 non-null   float64
 5    2022_2    4765 non-null   float64
 6    2022_1    4765 non-null   float64
 7    2022_0    4767 non-null   float64
 8    2021_4    4766 non-null   float64
 9    2021_3    4766 non-null   float64
 10   2021_2    4766 non-null   float64
 11   2021_1    4766 non-null   float64
 12   2021_0    4768 non-null   float64
 13   2020_4    4766 non-null   float64
 14   2020_3    4766 non-null   float64
 15   2020_2    4766 non-null   float64
 16   2020_1    4766 non-null   float64
 17   2020_0    4768 non-null   float64
 18   2019_4    4734 non-null   float64
 19   2019_3    4734 non-null   float64
 20   2019_2

In [11]:
rent_prices.shape

(4773, 198)

We can see that almost every column has a nonnull entry and we need to decide what to do with them. We can also see that the name columns have a lot of repeated values but we don't know what that means for the data yet.

First, let's deal with the null values.

In [12]:
rent_prices.columns[rent_prices.isnull().any()]
#results show that each column has at least one missing value.

Index(['pmsaname', 'cntyname', 'areaname', '2022_4', '2022_3', '2022_2',
       '2022_1', '2022_0', '2021_4', '2021_3',
       ...
       '1985_4', '1985_3', '1985_2', '1985_1', '1985_0', '1983_4', '1983_3',
       '1983_2', '1983_1', '1983_0'],
      dtype='object', length=198)

In [13]:
pd.options.display.min_rows = 198
rent_prices.isnull().sum()
#it looks like the number of null values per column is not a lot so they should be easy to deal with.
#to be sure we'll look at the largest number of null values per column

pmsaname    16
cntyname    16
areaname     7
2022_4       8
2022_3       8
2022_2       8
2022_1       8
2022_0       6
2021_4       7
2021_3       7
2021_2       7
2021_1       7
2021_0       5
2020_4       7
2020_3       7
2020_2       7
2020_1       7
2020_0       5
2019_4      39
2019_3      39
2019_2      39
2019_1      39
2019_0      37
2018_4      39
2018_3      39
2018_2      39
2018_1      39
2018_0      37
2017_4      39
2017_3      39
            ..
1989_4      22
1989_3      22
1989_2      22
1989_1      22
1989_0      20
1988_4      22
1988_3      22
1988_2      22
1988_1      22
1988_0      20
1987_4      22
1987_3      22
1987_2      22
1987_1      22
1987_0      20
1986_4      22
1986_3      22
1986_2      22
1986_1      22
1986_0      20
1985_4      35
1985_3      35
1985_2      35
1985_1      33
1985_0      35
1983_4      37
1983_3      37
1983_2      37
1983_1      37
1983_0      35
Length: 198, dtype: int64

In [14]:
rent_prices.isnull().sum().nlargest(30)
#we can see that we have over 50 null values for some years

2002_4    61
2002_3    61
2002_2    61
2002_1    61
2002_0    61
2001_4    60
2001_3    60
2001_2    60
2001_1    60
2001_0    60
2000_4    60
2000_3    60
2000_2    60
2000_1    60
2000_0    60
2006_4    58
2006_3    58
2006_2    58
2006_1    58
2005_4    58
2005_3    58
2005_2    58
2005_1    58
2005_0    58
2004_4    58
2004_3    58
2004_2    58
2004_1    58
2004_0    58
2003_4    58
dtype: int64

We cannot see all the information at a glance because we have 198 rows which is over pandas display limits.
Let's divide the null columns into ranges so we know what we're dealing with.

In [15]:
null_cols = rent_prices.columns[rent_prices.isnull().any()]
print(str(len([col for col in null_cols if rent_prices[col].isnull().sum()>50])) + ' columns with na >50')
print(str(len([col for col in null_cols if rent_prices[col].isnull().sum()>30
              if rent_prices[col].isnull().sum()<50])) + ' columns with 30<na<50')
print(str(len([col for col in null_cols if rent_prices[col].isnull().sum()>0
              if rent_prices[col].isnull().sum()<30])) + ' columns with 0<na<30')

50 columns with na >50
60 columns with 30<na<50
88 columns with 0<na<30


### Fix null values

For the name columns, we can delete all the columns that are blank in all 3 name columns, and replace the na values in the rest with each other. 

In [16]:
#Area name has the least null values so we will fill pmsaname and cntyname with areaname.
rent_prices = rent_prices.assign(pmsaname=lambda x: np.where(x.pmsaname.isnull(),x.areaname,x.pmsaname),
                   cntyname=lambda x: np.where(x.cntyname.isnull(),x.areaname,x.cntyname))
print(rent_prices.pmsaname.isnull().sum())
print(rent_prices.cntyname.isnull().sum())
#All the name columns now have equal number of missing values which makes me suspect that they are the same rows.
#We have no need for rows with no name columns so we will drop them, but first let's check them out

7
7


In [17]:
rent_prices[rent_prices.pmsaname.isnull()]
#let's see what's happening around each row

,pmsaname,cntyname,areaname,2022_4,2022_3,2022_2,2022_1,2022_0,2021_4,2021_3,...,1985_4,1985_3,1985_2,1985_1,1985_0,1983_4,1983_3,1983_2,1983_1,1983_0
2090,NaN,NaN,NaN,2990.0,2726.0,2205.0,1826.0,1658.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2149,NaN,NaN,NaN,2990.0,2726.0,2205.0,1826.0,1658.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2159,NaN,NaN,NaN,2505.0,2181.0,1723.0,1309.0,1145.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,935.0,NaN,NaN,...,NaN,NaN,NaN,559.0,NaN,NaN,NaN,NaN,NaN,489.0
4770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1081.4,NaN,NaN,...,NaN,NaN,NaN,569.4,NaN,NaN,NaN,NaN,NaN,540.8
4771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
rent_prices.loc[2088:2092,['pmsaname','cntyname','areaname','2022_4','2022_3']]
#we need to delete the rows that have no name as they are not significant to our data.
#we can also see that there are duplicated rows in the data across the entire columns and across the name columns

,pmsaname,cntyname,areaname,2022_4,2022_3
2088,"Boston, MA--NH PMSA",Middlesex County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2990.0,2726.0
2089,"Boston, MA--NH PMSA",Middlesex County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",NaN,NaN
2090,NaN,NaN,NaN,2990.0,2726.0
2091,"Lowell, MA--NH PMSA",Middlesex County,"Lowell, MA HUD Metro FMR Area",2404.0,2192.0
2092,"Boston, MA--NH PMSA",Middlesex County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2990.0,2726.0


In [19]:
rent_prices.loc[2147:2151,['pmsaname','cntyname','areaname','2022_4','2022_3']]
#same issue as above

,pmsaname,cntyname,areaname,2022_4,2022_3
2147,"Boston, MA--NH PMSA",Norfolk County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2990.0,2726.0
2148,"Boston, MA--NH PMSA",Norfolk County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",NaN,NaN
2149,NaN,NaN,NaN,2990.0,2726.0
2150,"Boston, MA--NH PMSA",Norfolk County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2990.0,2726.0
2151,"Boston, MA--NH PMSA",Norfolk County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2990.0,2726.0


In [20]:
rent_prices.loc[2157:2161,['pmsaname','cntyname','areaname','2022_4','2022_3']]
#same issue as above

,pmsaname,cntyname,areaname,2022_4,2022_3
2157,"Brockton, MA PMSA",Plymouth County,"Brockton, MA HUD Metro FMR Area",2505.0,2181.0
2158,"Brockton, MA PMSA",Plymouth County,"Brockton, MA HUD Metro FMR Area",NaN,NaN
2159,NaN,NaN,NaN,2505.0,2181.0
2160,"Brockton, MA PMSA",Plymouth County,"Brockton, MA HUD Metro FMR Area",2505.0,2181.0
2161,"Boston, MA--NH PMSA",Plymouth County,"Boston-Cambridge-Quincy, MA-NH HUD Metro FMR Area",2990.0,2726.0


In [21]:
rent_prices.loc[4767:4775,['pmsaname','cntyname','areaname','2022_4','2022_3']]
#all nan values

,pmsaname,cntyname,areaname,2022_4,2022_3
4767,"St. John/St. Thomas, VI",St. John,"St. John Island, VI",2305.0,2101.0
4768,"St. John/St. Thomas, VI",St. Thomas,"St. Thomas Island, VI",1699.0,1549.0
4769,NaN,NaN,NaN,NaN,NaN
4770,NaN,NaN,NaN,NaN,NaN
4771,NaN,NaN,NaN,NaN,NaN
4772,NaN,NaN,NaN,NaN,NaN


In [22]:
rent_prices = rent_prices.drop([2090,2149,2159,4769,4770,4771,4772])
rent_prices.tail(3)
#we drop the rows that have null values across all name columns

,pmsaname,cntyname,areaname,2022_4,2022_3,2022_2,2022_1,2022_0,2021_4,2021_3,...,1985_4,1985_3,1985_2,1985_1,1985_0,1983_4,1983_3,1983_2,1983_1,1983_0
4766,"St. Croix, VI",St. Croix,"St. Croix Island, VI",1467.0,1338.0,1082.0,886.0,868.0,1410.0,1294.0,...,833.0,728.0,583.0,481.0,462.0,1019.0,909.0,729.0,618.0,509.0
4767,"St. John/St. Thomas, VI",St. John,"St. John Island, VI",2305.0,2101.0,1700.0,1368.0,1154.0,2214.0,2031.0,...,1046.0,1001.0,808.0,628.0,525.0,1307.0,1167.0,934.0,792.0,654.0
4768,"St. John/St. Thomas, VI",St. Thomas,"St. Thomas Island, VI",1699.0,1549.0,1253.0,1001.0,832.0,1633.0,1498.0,...,1046.0,1001.0,808.0,628.0,525.0,1307.0,1167.0,934.0,792.0,654.0


Now it's time to delete duplicate values across the rows

In [23]:
rent_prices[rent_prices.duplicated()].shape[0]
#shows the number of duplicated rows

1470

In [24]:
rent_prices[rent_prices.duplicated(subset=['pmsaname','cntyname','areaname'])].shape[0]
#however the names are duplicated 7 more times and we need to delete those as well

1477

In [25]:
rent_prices = rent_prices.drop_duplicates(subset=['pmsaname','cntyname','areaname'])
rent_prices.shape
#we now have 3289 rows
#now let's reanalyze the na values.
#in hindsight, this should have been done first.

(3289, 198)

For the rent prices information, missing data means that US HUDS was unable to get the data from that region or that the data does not exist. We have at most 61 missing values out of 4773. We can replace these missing values with 0. However, doing that will make calculations for average rent prices wrong. 

We will use a fillna method. However, we need to ensure that the fillna method only takes from the same year, otherwise average rent price calculations will be wrong.


In [26]:
coln = rent_prices.columns
for i in coln:
    if str(i[-1])=='0':
        rent_prices.fillna(method='ffill',axis=1)
    else:
        rent_prices.fillna(method='bfill',axis=1)
rent_prices.isnull().sum()
#this function ensures that fill values only belong to that year

pmsaname     0
cntyname     0
areaname     0
2022_4       0
2022_3       0
2022_2       0
2022_1       0
2022_0       0
2021_4       0
2021_3       0
2021_2       0
2021_1       0
2021_0       0
2020_4       0
2020_3       0
2020_2       0
2020_1       0
2020_0       0
2019_4      30
2019_3      30
2019_2      30
2019_1      30
2019_0      30
2018_4      30
2018_3      30
2018_2      30
2018_1      30
2018_0      30
2017_4      30
2017_3      30
            ..
1989_4      13
1989_3      13
1989_2      13
1989_1      13
1989_0      13
1988_4      13
1988_3      13
1988_2      13
1988_1      13
1988_0      13
1987_4      13
1987_3      13
1987_2      13
1987_1      13
1987_0      13
1986_4      13
1986_3      13
1986_2      13
1986_1      13
1986_0      13
1985_4      26
1985_3      26
1985_2      26
1985_1      26
1985_0      26
1983_4      28
1983_3      28
1983_2      28
1983_1      28
1983_0      28
Length: 198, dtype: int64

The function was not enough to completely remove all na values because some towns have no data available for a full year. We need to decide what to do in this case. 

We will use ffill for the remaining na values so that rent price is kept constant for areas where information is missing. We could've done this for all other na values as well since total number is not very significant for analysis. However, been very detailed is essential at this stage of learning experience.

In [27]:
rent_prices = rent_prices.fillna(method='ffill',axis=1)
rent_prices.isnull().sum()
#no more null values.

pmsaname    0
cntyname    0
areaname    0
2022_4      0
2022_3      0
2022_2      0
2022_1      0
2022_0      0
2021_4      0
2021_3      0
2021_2      0
2021_1      0
2021_0      0
2020_4      0
2020_3      0
2020_2      0
2020_1      0
2020_0      0
2019_4      0
2019_3      0
2019_2      0
2019_1      0
2019_0      0
2018_4      0
2018_3      0
2018_2      0
2018_1      0
2018_0      0
2017_4      0
2017_3      0
           ..
1989_4      0
1989_3      0
1989_2      0
1989_1      0
1989_0      0
1988_4      0
1988_3      0
1988_2      0
1988_1      0
1988_0      0
1987_4      0
1987_3      0
1987_2      0
1987_1      0
1987_0      0
1986_4      0
1986_3      0
1986_2      0
1986_1      0
1986_0      0
1985_4      0
1985_3      0
1985_2      0
1985_1      0
1985_0      0
1983_4      0
1983_3      0
1983_2      0
1983_1      0
1983_0      0
Length: 198, dtype: int64

### Reshape Data

To make meaning of our data, we need to e need to group our data by year and bedroom types.

We can start by creating a new row for the bedroom types based on the value of the column names. 

In [28]:
coln = rent_prices.columns
bedrm = []
for i in coln:
    if str(i[-1])=='0':
        bedrm.append('Efficiency')
    elif str(i[-1])=='1':
        bedrm.append('1-bed')
    elif str(i[-1])=='2':
        bedrm.append('2-bed')
    elif str(i[-1])=='3':
        bedrm.append('3-bed')
    elif str(i[-1])=='4':
        bedrm.append('4-bed')
    elif str(i[-1])=='e':
        bedrm.append('Room type')
rent_prices.loc[-1] = bedrm
rent_prices.index = rent_prices.index + 1
rent_prices = rent_prices.sort_index()
#above functions create a new row 'room type' just below the columns

Now that we have the room type column, we can take out the type code from the year values.

In [29]:
new_coln = ['pmsaname','cntyname','areaname']+[col[0:4] for col in rent_prices.columns if not col.endswith('name')]
rent_prices.rename(columns = dict(zip(coln,new_coln)),inplace=True)
rent_prices.head(1)

,pmsaname,cntyname,areaname,2022,2022,2022,2022,2022,2021,2021,...,1985,1985,1985,1985,1985,1983,1983,1983,1983,1983
0,Room type,Room type,Room type,4-bed,3-bed,2-bed,1-bed,Efficiency,4-bed,3-bed,...,4-bed,3-bed,2-bed,1-bed,Efficiency,4-bed,3-bed,2-bed,1-bed,Efficiency


Our data is complicated because it has several possible indexes: year, bedroom type, pmsaname, cntyname, areaname. We have to reshape in such a way that this is effectively communicated.

We will need to have multiindexes on both rows and columns

In [30]:
rent_prices = rent_prices.T
rent_prices.set_index([rent_prices.index,rent_prices[0]],inplace=True)
rent_prices.drop([0],axis=1,inplace=True)
rent_prices.head(10)
#this creates a row index with year and bedroom type

1                              2     \
         0                                                               
pmsaname Room type   Montgomery, AL MSA                 Mobile, AL MSA   
cntyname Room type       Autauga County                 Baldwin County   
areaname Room type   Montgomery, AL MSA  Daphne-Fairhope-Foley, AL MSA   
2022     4-bed                   1494.0                         1715.0   
         3-bed                   1156.0                         1348.0   
         2-bed                    914.0                         1017.0   
         1-bed                    764.0                          777.0   
         Efficiency               643.0                          772.0   
2021     4-bed                   1520.0                         1584.0   
         3-bed                   1148.0                         1249.0   

                                   3     \
         0                                
pmsaname Room type   Barbour County, AL   
cntyname Room type       Barbour County   
areaname Room type   Barbour County, AL   
2022     4-bed                    980.0   
         3-bed                    871.0   
         2-bed                    705.0   
         1-bed                    536.0   
         Efficiency               532.0   
2021     4-bed                    907.0   
         3-bed                    806.0   

                                                         4     \
         0                                                      
pmsaname Room type                            Bibb County, AL   
cntyname Room type                                Bibb County   
areaname Room type   Birmingham-Hoover, AL HUD Metro FMR Area   
2022     4-bed                                         1316.0   
         3-bed                                         1220.0   
         2-bed                                          943.0   
         1-bed                                          820.0   
         Efficiency                                     765.0   
2021     4-bed                                         1409.0   
         3-bed                                         1303.0   

                                                         5     \
         0                                                      
pmsaname Room type                         Birmingham, AL MSA   
cntyname Room type                              Blount County   
areaname Room type   Birmingham-Hoover, AL HUD Metro FMR Area   
2022     4-bed                                         1316.0   
         3-bed                                         1220.0   
         2-bed                                          943.0   
         1-bed                                          820.0   
         Efficiency                                     765.0   
2021     4-bed                                         1409.0   
         3-bed                                         1303.0   

                                   6                  7     \
         0                                                   
pmsaname Room type   Bullock County, AL  Butler County, AL   
cntyname Room type       Bullock County      Butler County   
areaname Room type   Bullock County, AL  Butler County, AL   
2022     4-bed                   1158.0             1043.0   
         3-bed                    968.0              871.0   
         2-bed                    783.0              705.0   
         1-bed                    602.0              619.0   
         Efficiency               589.0              530.0   
2021     4-bed                   1034.0              930.0   
         3-bed                    877.0              789.0   

                                                     8     \
         0                                                  
pmsaname Room type                       Anniston, AL MSA   
cntyname Room type                         Calhoun County   
areaname Room type   Anniston-Oxford-Jacksonville, AL MSA   
2022     4-bed   

In [31]:
rent_prices.columns = [rent_prices.iloc[0,:].tolist(),rent_prices.iloc[1,:].tolist(),rent_prices.iloc[2,:].tolist()]
rent_prices.head(10)
#this creates a column index with the town names
#however, we have duplicate rows which we will proceed to delete

Montgomery, AL MSA                 Mobile, AL MSA  \
                         Autauga County                 Baldwin County   
                     Montgomery, AL MSA  Daphne-Fairhope-Foley, AL MSA   
         0                                                               
pmsaname Room type   Montgomery, AL MSA                 Mobile, AL MSA   
cntyname Room type       Autauga County                 Baldwin County   
areaname Room type   Montgomery, AL MSA  Daphne-Fairhope-Foley, AL MSA   
2022     4-bed                   1494.0                         1715.0   
         3-bed                   1156.0                         1348.0   
         2-bed                    914.0                         1017.0   
         1-bed                    764.0                          777.0   
         Efficiency               643.0                          772.0   
2021     4-bed                   1520.0                         1584.0   
         3-bed                   1148.0                         1249.0   

                     Barbour County, AL  \
                         Barbour County   
                     Barbour County, AL   
         0                                
pmsaname Room type   Barbour County, AL   
cntyname Room type       Barbour County   
areaname Room type   Barbour County, AL   
2022     4-bed                    980.0   
         3-bed                    871.0   
         2-bed                    705.0   
         1-bed                    536.0   
         Efficiency               532.0   
2021     4-bed                    907.0   
         3-bed                    806.0   

                                              Bibb County, AL  \
                                                  Bibb County   
                     Birmingham-Hoover, AL HUD Metro FMR Area   
         0                                                      
pmsaname Room type                            Bibb County, AL   
cntyname Room type                                Bibb County   
areaname Room type   Birmingham-Hoover, AL HUD Metro FMR Area   
2022     4-bed                                         1316.0   
         3-bed                                         1220.0   
         2-bed                                          943.0   
         1-bed                                          820.0   
         Efficiency                                     765.0   
2021     4-bed                                         1409.0   
         3-bed                                         1303.0   

                                           Birmingham, AL MSA  \
                                                Blount County   
                     Birmingham-Hoover, AL HUD Metro FMR Area   
         0                                                      
pmsaname Room type                         Birmingham, AL MSA   
cntyname Room type                              Blount County   
areaname Room type   Birmingham-Hoover, AL HUD Metro FMR Area   
2022     4-bed                                         1316.0   
         3-bed                                         1220.0   
         2-bed                                          943.0   
         1-bed                                          820.0   
         Efficiency                                     765.0   
2021     4-bed                                         1409.0   
         3-bed                                         1303.0   

                     Bullock County, AL  Butler County, AL  \
                         Bullock County      Butler County   
                     Bullock County, AL  Butler County, AL   
         0                                                   
pmsaname Room type   Bullock County, AL  Butler County, AL   
cntyname Room type       Bullock County      Butler County   
areaname Room type   Bullock County, AL  Butler County, AL   
2022     4-bed                   1158.0             1043.0   
         3-bed                    968.0              871.0   
 

In [32]:
rent_prices = rent_prices.drop([('pmsaname',  'Room type'),
            ('cntyname',  'Room type'),
            ('areaname',  'Room type')])
rent_prices.head(10)

Montgomery, AL MSA                Mobile, AL MSA  \
                    Autauga County                Baldwin County   
                Montgomery, AL MSA Daphne-Fairhope-Foley, AL MSA   
     0                                                             
2022 4-bed                  1494.0                        1715.0   
     3-bed                  1156.0                        1348.0   
     2-bed                   914.0                        1017.0   
     1-bed                   764.0                         777.0   
     Efficiency              643.0                         772.0   
2021 4-bed                  1520.0                        1584.0   
     3-bed                  1148.0                        1249.0   
     2-bed                   908.0                         922.0   
     1-bed                   766.0                         723.0   
     Efficiency              640.0                         718.0   

                Barbour County, AL                          Bibb County, AL  \
                    Barbour County                              Bibb County   
                Barbour County, AL Birmingham-Hoover, AL HUD Metro FMR Area   
     0                                                                        
2022 4-bed                   980.0                                   1316.0   
     3-bed                   871.0                                   1220.0   
     2-bed                   705.0                                    943.0   
     1-bed                   536.0                                    820.0   
     Efficiency              532.0                                    765.0   
2021 4-bed                   907.0                                   1409.0   
     3-bed                   806.0                                   1303.0   
     2-bed                   648.0                                   1002.0   
     1-bed                   492.0                                    871.0   
     Efficiency              488.0                                    817.0   

                                      Birmingham, AL MSA Bullock County, AL  \
                                           Blount County     Bullock County   
                Birmingham-Hoover, AL HUD Metro FMR Area Bullock County, AL   
     0                                                                        
2022 4-bed                                        1316.0             1158.0   
     3-bed                                        1220.0              968.0   
     2-bed                                         943.0              783.0   
     1-bed                                         820.0              602.0   
     Efficiency                                    765.0              589.0   
2021 4-bed                                        1409.0             1034.0   
     3-bed                                        1303.0              877.0   
     2-bed                                        1002.0              705.0   
     1-bed                                         871.0              549.0   
     Efficiency                                    817.0              532.0   

                Butler County, AL                     Anniston, AL MSA  \
                    Butler County                       Calhoun County   
                Butler County, AL Anniston-Oxford-Jacksonville, AL MSA   
     0                                                                   
2022 4-bed                 1043.0                               1111.0   
     3-bed                  871.0                                988.0   
     2-bed                  705.0                                744.0   
     1-bed                  619.0                                565.0   
     Efficiency             530.0                                562.0   
2021 4-bed                  930.0                               1094.0   
     3-bed                  789.0                                944.0   
     2-bed                  634.0  

In [33]:
rent_prices.index.set_names(['Year', 'Roomtype'], inplace=True)
rent_prices.head(10)
#we set names for the index so our data looks neat

Montgomery, AL MSA                Mobile, AL MSA  \
                    Autauga County                Baldwin County   
                Montgomery, AL MSA Daphne-Fairhope-Foley, AL MSA   
Year Roomtype                                                      
2022 4-bed                  1494.0                        1715.0   
     3-bed                  1156.0                        1348.0   
     2-bed                   914.0                        1017.0   
     1-bed                   764.0                         777.0   
     Efficiency              643.0                         772.0   
2021 4-bed                  1520.0                        1584.0   
     3-bed                  1148.0                        1249.0   
     2-bed                   908.0                         922.0   
     1-bed                   766.0                         723.0   
     Efficiency              640.0                         718.0   

                Barbour County, AL                          Bibb County, AL  \
                    Barbour County                              Bibb County   
                Barbour County, AL Birmingham-Hoover, AL HUD Metro FMR Area   
Year Roomtype                                                                 
2022 4-bed                   980.0                                   1316.0   
     3-bed                   871.0                                   1220.0   
     2-bed                   705.0                                    943.0   
     1-bed                   536.0                                    820.0   
     Efficiency              532.0                                    765.0   
2021 4-bed                   907.0                                   1409.0   
     3-bed                   806.0                                   1303.0   
     2-bed                   648.0                                   1002.0   
     1-bed                   492.0                                    871.0   
     Efficiency              488.0                                    817.0   

                                      Birmingham, AL MSA Bullock County, AL  \
                                           Blount County     Bullock County   
                Birmingham-Hoover, AL HUD Metro FMR Area Bullock County, AL   
Year Roomtype                                                                 
2022 4-bed                                        1316.0             1158.0   
     3-bed                                        1220.0              968.0   
     2-bed                                         943.0              783.0   
     1-bed                                         820.0              602.0   
     Efficiency                                    765.0              589.0   
2021 4-bed                                        1409.0             1034.0   
     3-bed                                        1303.0              877.0   
     2-bed                                        1002.0              705.0   
     1-bed                                         871.0              549.0   
     Efficiency                                    817.0              532.0   

                Butler County, AL                     Anniston, AL MSA  \
                    Butler County                       Calhoun County   
                Butler County, AL Anniston-Oxford-Jacksonville, AL MSA   
Year Roomtype                                                            
2022 4-bed                 1043.0                               1111.0   
     3-bed                  871.0                                988.0   
     2-bed                  705.0                                744.0   
     1-bed                  619.0                                565.0   
     Efficiency             530.0                                562.0   
2021 4-bed                  930.0                               1094.0   
     3-bed                  789.0                                944.0   
     2-bed                  634.0  

Because we have text data above the year index (in the column index) we cannot convert year to datetime. During analysis we will have to section off our data if we want to use datetime functions.

### Next Steps...

We are comparing rent prices data to university enrollment data in each of the town. We need to import university enrollment data and clean it up, then merge both dataframes for analysis.